In [21]:
import operator
import os
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition
from langgraph.types import Command
from langgraph.graph import MessagesState, add_messages
from pydantic import BaseModel, Field
from typing import Annotated, Sequence, Optional, Dict, Any, TypedDict, Literal
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import PromptTemplate

### Initializing the LLMs

In [2]:
## Langchain and Langsmith tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ["LANGCHAIN_TRACING_V2"]="true"

## Getting Froq API key
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# EMBEDDING_MODEL = "BAAI/bge-small-en"
GROQ_INFERENCE_MODEL = "deepseek-r1-distill-llama-70b"
OPENAI_INFERENCE_MODEL = "gpt-4o"
# INFERENCE_MODEL = "gemma2-9b-it"

In [3]:
llm = ChatGroq(model=GROQ_INFERENCE_MODEL, temperature=0.3)
print('--------Creating the response from llm based on contexts--------')

# llm = ChatOpenAI(model=OPENAI_INFERENCE_MODEL, temperature=0.3)
# print('--------Creating the response from llm based on contexts--------')

--------Creating the response from llm based on contexts--------


### State of the Agents

In [ ]:
class SupervisorState(MessagesState):
    """State of the Supervisor Agent"""
    query: str
    research_results: Optional[Dict[str, Any]] = None
    summary_results: Optional[Dict[str, Any]] = None
    final_documents: Optional[str] = None
    next: Optional[Literal['research', 'report', 'FINISH']] = None
    reasoning: Optional[str] = None
    task_details: Optional[str] = None


In [14]:
class ResearchState(MessagesState):
    """State of the Team 1: Research Agent"""
    query: str
    next: Optional[Literal['finance', 'pharma']] = None
    reasoning: Optional[str] = None
    task_details: Optional[str] = None
    research_context: str
    medical_results: Optional[Dict[str, Any]]
    financial_results: Optional[Dict[str, Any]]

In [26]:
class ReportingState(MessagesState):
    """State of the Team 2: Reporting Agent"""
    query: str
    next: Optional[Literal['summarization', 'doc_generation']] = None
    reasoning: str
    task_details: str
    research_findings: str = ""
    report_type: Optional[Literal['summary','document']] = None
    summary: Optional[str]
    document_content: str = ""
    has_summary: bool = False
    # report_format:Optional[Literal['pdf','docx','html','markdown']] = None
    # research_data: Dict[str, Any]
    report_context: str = ""
    report_requirements: str = ""
    final_report: Optional[str]

### Router of the Agents

In [6]:
class SupervisorRouter(TypedDict):
    next: Literal['research', 'report', 'FINISH']
    reasoning: str
    task_details: str

In [19]:
class ResearchRouter(TypedDict):
    next: Literal['pharma', 'financial']
    reasoning: str
    task_details: str
    research_context: str

In [25]:
class ReportRouter(TypedDict):
    next: Literal['summary', 'document']
    reasoning: str
    report_requirements: str
    report_context: str
    report_format: str

### Agent Prompt Template

#### supervisor prompt template

In [7]:
supervisor_prompt_template = """You are the Supervisor Agent coordinating a hierarchical multi-agent research system.

Your responsibilities:
1. Analyze incoming requests to determine if research or reporting is needed
2. Coordinate between Team 1 (Research) and Team 2 (Reporting)
3. Make decisions on workflow progression

Teams under your supervision:
- Team 1 (Research): Coordinates medical/pharma (Team 3) and financial (Team 4) research
- Team 2 (Reporting): Coordinates summary creation (Team 5) and document generation (Team 6)

Decision criteria:
- If no research results exist and task requires data gathering then route to "research" agent i.e. Team 1
- If research results exist but no report generated then route to "reporting" agent i.e. Team 2  
- If final document is complete then route to "end"

Always respond with your decision and reasoning in JSON format:
{{  "next_action": "research|reporting|end", 
    "reasoning": "explanation", 
    "task_details": "specific instructions"}}

Here is the user query: {question}
"""

In [9]:
supervisor_llm_with_structured_output = llm.with_structured_output(SupervisorRouter)
formatted_prompt = supervisor_prompt_template.format(question="What are the components of the medicine Ecosprin 75?")
response = supervisor_llm_with_structured_output.invoke(formatted_prompt)

In [10]:
response

{'next': 'research',
 'reasoning': "The user is asking about the components of Ecosprin 75, which requires gathering data on the medicine's ingredients. Since this is a data-gathering task and no prior research results are mentioned, it should be routed to the research team for further investigation.",
 'task_details': 'Investigate and gather information on the components of Ecosprin 75.'}

#### researcher prompt template

In [ ]:
researcher_prompt_template = """You are a research coordinator that determines which specialized research agent should handle a query.

Your responsibilities:
1. Analyze the incoming query and determine whether it requires:
    - "finance": Financial research, market analysis, economic data, investment research, financial modeling, etc.
    - "pharma": Pharmaceutical research, drug development, clinical trials, regulatory affairs, medical research, etc.
2. Coordinate between Team 3 (Pharma Specialist Researcher) and Team 4 (Finance Specialist Researcher)
3. Consolidate results from specialized research
4. Report back to Supervisor

Teams under your supervision:
- Team 3 (Pharma Specialist Researcher): Perform Pharma Related Research and returns query related research information in the field of pharma and medicine
- Team 4 (Finance Specialist Researcher): Perform Finance Related Research and returns query related research information in the field of finance

Consider the following:
- What type of specialized knowledge is required?
- Which domain expertise would be most valuable?
- Are there any domain-specific terminologies or concepts?
- What kind of data sources would be most relevant?
    
Provide your routing decision with clear reasoning and specific task details for the chosen agent.

Query: {question}
Task Details: {task_details}
"""

In [20]:
researcher_llm_with_structured_output = llm.with_structured_output(ResearchRouter)
research_formatted_prompt = researcher_prompt_template.format(question="What are the components of the medicine Ecosprin 75?",
                                                     task_details="Investigate and gather information on the components of Ecosprin 75.")
response = researcher_llm_with_structured_output.invoke(research_formatted_prompt)
response

{'next': 'pharma',
 'reasoning': 'The query is about the components of Ecosprin 75, which is a pharmaceutical product. Therefore, it requires expertise in pharmaceutical research to identify the active and inactive ingredients.',
 'research_context': 'Ecosprin 75 is a medication, and the query is about its components, which falls under pharmaceutical research.',
 'task_details': 'Investigate and gather information on the components of Ecosprin 75, including active and inactive ingredients.'}

#### reporter prompt template

In [28]:
reporter_prompt_template = """You are a report coordinator that determines the appropriate reporting workflow based on the research findings and requirements.

Analyze the request and determine the reporting workflow:

DECISION RULES:
- If NO summary exists AND user wants a document: Route to "summary" first
- If summary exists AND user wants a document: Route to "document" 
- If user only wants a summary: Route to "summary"
- If unclear, default to "summary" first

Consider the following factors:
- What type of output does the user need? (summary, full document, both)
- What format is most appropriate? (markdown, PDF, HTML, etc.)
- How detailed should the report be?
- What is the intended audience?
- Are there specific requirements mentioned?

Provide your routing decision with clear reasoning and specific requirements for the chosen agent.

Query: {question}
Task Details: {task_details}
Research Findings: {research_findings}
"""

In [29]:
reporter_llm_with_structured_output = llm.with_structured_output(ReportRouter)
reporter_formatted_prompt = reporter_prompt_template.format(question="What are the components of the medicine Ecosprin 75?",
                                                     task_details="Investigate and gather information on the components of Ecosprin 75.",
                                                     research_findings="Ecosprin 75 helps in keeping blood thinner")
response = reporter_llm_with_structured_output.invoke(reporter_formatted_prompt)
response

{'next': 'document',
 'reasoning': "The user is asking for detailed information about the components of Ecosprin 75, which suggests they need a comprehensive report rather than a brief summary. The research findings indicate that Ecosprin 75 is used to keep blood thinner, which is a specific use case that warrants a detailed explanation. Therefore, routing to 'document' is appropriate to provide in-depth information on the components and their effects.",
 'report_context': 'The user is inquiring about the components of Ecosprin 75, which is a medication used to thin blood. This suggests the user may be a healthcare professional or a patient seeking detailed information about their medication.',
 'report_format': 'markdown',
 'report_requirements': 'Detailed explanation of components and their effects.'}

### Agents

#### Supervisor Agent

In [ ]:
def supervisor(state:SupervisorState)->Command[Literal['research', 'report', '__end__']]:
    """This is the supervisor agent. It takes the query and redirects it to either the research node or reporting node"""
    supervisor_llm_with_structured_output = llm.with_structured_output(SupervisorRouter)
    formatted_prompt = supervisor_prompt_template.format(question=state["query"])
    response = supervisor_llm_with_structured_output.invoke(formatted_prompt)
    
    goto = response["next"]

    if goto == "FINISH":
        goto=END
    
    supervisor_message = AIMessage(
        content=f"SUPERVISOR DECISION: Routing to {goto}. Reasoning: {response['reasoning']}. Task Details: {response['task_details']}"
    )
    
    return Command(goto=goto, update={
        "query": state["query"],
        "next": goto,
        "reasoning": response["reasoning"],
        "task_details": response["task_details"],
        "messages": [supervisor_message]})
    

#### Researcher Agent

In [ ]:
def researcher(state:ResearchState)->Command[Literal['pharma','finance']]:
    """This is the supervisor agent. It takes the query from the supervisor agent, analyses whether it is related to pharma or finance and redirects it to either to pharma agent and finance agent."""
    query = state["query"]
    task_details = state["task_details"]
    researcher_llm_with_structured_output = llm.with_structured_output(ResearchRouter)
    formatted_prompt = researcher_prompt_template.format(question=query, task_details=task_details)
    response = researcher_llm_with_structured_output.invoke(formatted_prompt)

    goto = response["next"]

    if goto == "FINISH":
        goto=END
    
    researcher_message = AIMessage(
        content=f"RESEARCHER DECISION: Routing to {goto}. Reasoning: {response['reasoning']}. Research Context: {response['research_context']}. Task Details: {response['task_details']}"
    )

    return Command(goto=goto, update={
        "query": state["query"],
        "next": goto,
        "reasoning": response["reasoning"],
        "task_details": response["task_details"],
        "research_context": response["research_context"],
        "messages": add_messages([researcher_message])})
    

#### Reporter Agent

In [32]:
def reporter(state:ReportingState):
    """This is the supervisor agent. It takes the query from the supervisor agent, analyses whether it is related to pharma or finance and redirects it to either to pharma agent and finance agent."""
    query = state["query"]
    task_details = state["task_details"]
    research_findings = state["research_findings"]
    reporter_llm_with_structured_output = llm.with_structured_output(ReportRouter)
    reporter_formatted_prompt = reporter_prompt_template.format(question=query,
                                                     task_details=task_details,
                                                     research_findings=research_findings
                                                     )
    response = reporter_llm_with_structured_output.invoke(reporter_formatted_prompt)
    
    goto = response["next"]

    if goto == "FINISH":
        goto=END
    
    reporter_message = AIMessage(
        content=f"REPORTER DECISION: Routing to {goto}. Reasoning: {response['reasoning']}. Report Requirements: {response['report_requirements']}. Report Fomat: {response['report_format']}"
    )

    return Command(goto=goto, update={
        "query": state["query"],
        "next": goto,
        "reasoning": response["reasoning"],
        "task_details": response["task_details"],
        "research_context": response["research_context"],
        "research_findings": state.get("research_findings", ""),
        "report_type": goto,
        "report_context": response["report_context"],
        "report_requirements": response["report_requirements"],
        "report_format": response["report_format"],
        "messages": add_messages([reporter_message])})

#### Medical Researcher Agent

In [9]:
def medical_researcher(state:AgentState):
    pass

#### Finance Researcher Agent

In [10]:
def finance_researcher(state:AgentState):
    pass

#### Document Summarizer Agent

In [11]:
def document_summarizer(state:AgentState):
    pass

#### Document Generator Agent

In [12]:
def document_generator(state:AgentState):
    pass

### Workflow

In [14]:
graph_builder = StateGraph(AgentState)
graph_builder.add_node("Researcher", researcher)
graph_builder.add_node("Reporter", reporter)
graph_builder.add_node("Supervisor",supervisor)
graph_builder.add_node("Medical Researcher", medical_researcher)
graph_builder.add_node("Finance Researcher", finance_researcher)
graph_builder.add_node("Document Summarizer", document_summarizer)
graph_builder.add_node("Document Generator", document_generator)
graph_builder.add_edge(START, "Supervisor")
# graph_builder.add_conditional_edges("ai_assistant",
#                                     tools_condition)
# graph_builder.add_edge("tools","ai_assistant")
# app_react = graph_builder.compile(checkpointer=memory, interrupt_before=["tools"])